In [13]:
import pandas as pd
import numpy as np

df = pd.read_csv('clients_data.csv')
df.head(5)

,client_id,date,sessions,active_users,revenue,platform_type,country,bounce_rate,page_views
0,8270,2024-01-01,4200,957,2297.75,Web,BR,69.1,100
1,1860,2024-01-02,4945,82,199.53,Web,CA,62.0,68
2,6390,2024-01-03,4923,408,3731.84,Web,JP,21.7,36
3,6191,2024-01-04,1531,1438,2751.12,Tablet,US,18.4,146
4,6734,2024-01-05,1951,139,298.13,Web,FR,14.7,22


# Filtrando clientes inativos
Seguindo como base alguns critérios vamos filtrar clientes ativos como os que possuem datas de relatórios feitas no ano de 2025

In [28]:
df_clientes_ativos = df[df['date'] > '2024-12-31']
df_clientes_ativos.head(5)

,client_id,date,sessions,active_users,revenue,platform_type,country,bounce_rate,page_views
366,2150,2025-01-01,3379,2107,462.70,Tablet,FR,18.3,7
367,8560,2025-01-02,638,1804,172.76,Web,UK,13.2,4
368,2931,2025-01-03,2062,982,297.94,Web,US,40.7,90
369,2664,2025-01-04,1597,1652,2227.38,Web,UK,69.1,60
370,2081,2025-01-05,344,1895,1384.98,Web,CA,28.4,55


In [29]:
# Verifica duplicatas de clientes
coluna_chave = ["client_id"]  # Substitua pelas colunas relevantes
duplicados = df[df.duplicated(subset=coluna_chave, keep=False)]

if not duplicados.empty:
    print(f"⚠️ {len(duplicados)} registros duplicados nas colunas {coluna_chave}:")
    display(duplicados.sort_values(coluna_chave))
else:
    print(f"✅ Nenhuma duplicata encontrada nas colunas {coluna_chave}.")

⚠️ 104 registros duplicados nas colunas ['client_id']:


,client_id,date,sessions,active_users,revenue,platform_type,country,bounce_rate,page_views
594,1009,2025-08-17,2829,990,2822.66,Tablet,CA,29.5,9
811,1009,2026-03-22,1888,593,2054.60,Web,US,47.0,89
464,1698,2025-04-09,1100,538,3145.89,Mobile,JP,59.1,15
163,1698,2024-06-12,4112,2407,656.71,Web,BR,32.1,75
439,1728,2025-03-15,3561,2078,1557.32,Tablet,JP,24.6,109
...,...,...,...,...,...,...,...,...,...
953,9666,2026-08-11,4775,1311,2079.93,Mobile,JP,26.5,57
612,9702,2025-09-04,1319,1099,3141.16,Tablet,US,10.4,68
515,9702,2025-05-30,4094,1824,601.28,Mobile,US,45.1,43
527,9905,2025-06-11,477,914,363.75,Web,FR,70.4,142


# Agrupando clientes - Análise segmentada
Como vimos acima existem dados duplicados, porém as datas e o tipo de plataforma é diferente, logo, se agruparmos agora perderemos dados. A próxima etapa é separar os dados em  grupos de plataformas e depois fazer os estudos. 
Exemplo: grupo A (tablet), grupo B(web), grupo C(mobile).

In [32]:
# dividindo os grupos
df_tablet = df_clientes_ativos[df_clientes_ativos['platform_type'] == 'Tablet']
df_web = df_clientes_ativos[df_clientes_ativos['platform_type'] == 'Web']
df_mobile = df_clientes_ativos[df_clientes_ativos['platform_type'] == 'Mobile']

# Eliminando outliers
É de suma importancia eliminar dados que ''fogem da realidade'', então vamos isolar alguns casos no qual chamamos de outliers, já que será feita uma análise estatística esse tipo de dado pode interferir nos resultados finais.

Método utilizado: IQR - Uso do intervalo interquartil
é uma medida estatística que descreve a dispersão dos dados, ou seja, a distância entre o primeiro quartil (Q1) e o terceiro quartil (Q3) de um conjunto de dados
IQR=Q3−Q1

In [37]:
pd.set_option('display.width', 1000)  # Um valor bem alto evita a quebra
Q1 = df_tablet['revenue'].quantile(0.25)
Q3 = df_tablet['revenue'].quantile(0.75)
IQR = Q3 - Q1

limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

df_tablet_sem_outliers = df_tablet[(df_tablet['revenue'] >= limite_inferior) & (df_tablet['revenue'] <= limite_superior)]

print('Dados sem outliers')
df_tablet_sem_outliers.head(5)

Dados sem outliers


,client_id,date,sessions,active_users,revenue,platform_type,country,bounce_rate,page_views
366,2150,2025-01-01,3379,2107,462.70,Tablet,FR,18.3,7
374,1559,2025-01-09,1186,1459,2873.47,Tablet,FR,50.4,79
375,4672,2025-01-10,2114,556,2083.86,Tablet,UK,50.0,33
377,4343,2025-01-12,4845,590,4512.16,Tablet,UK,32.1,145
379,1580,2025-01-14,4820,1966,4872.12,Tablet,US,35.7,57


In [39]:
df_tablet_sem_outliers = df_tablet_sem_outliers.sort_values(by='revenue', ascending=False)
df_tablet_sem_outliers

,client_id,date,sessions,active_users,revenue,platform_type,country,bounce_rate,page_views
825,8984,2026-04-05,3660,1885,4946.20,Tablet,JP,76.0,58
663,5637,2025-10-25,2941,1760,4938.42,Tablet,US,19.8,38
764,8943,2026-02-03,3762,450,4931.37,Tablet,US,62.6,109
552,7190,2025-07-06,459,1557,4910.43,Tablet,FR,34.0,28
379,1580,2025-01-14,4820,1966,4872.12,Tablet,US,35.7,57
...,...,...,...,...,...,...,...,...,...
941,8112,2026-07-30,2543,1741,78.36,Tablet,US,13.9,61
734,1918,2026-01-04,1036,415,76.12,Tablet,DE,26.9,141
563,8114,2025-07-17,3688,1539,67.29,Tablet,UK,29.7,98
607,7938,2025-08-30,4440,940,43.34,Tablet,DE,77.7,79
